In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from chains import EthereumProcessor, EthereumQuerier
import json
import web3


In [2]:
sql_db = SQLDatabase()
querier = EthereumQuerier()
mongo_db = MongoDatabase()
processor = EthereumProcessor(sql_db, mongo_db, querier)



In [3]:
query = "SELECT * FROM evm_contract_abis WHERE network = 'Ethereum'"
sql_db.cursor.execute(query)
contract_abis = sql_db.cursor.fetchall()

In [4]:
contract_abis[100]

RealDictRow([('network', 'Ethereum'),
             ('contract_address',
              '0x254432E7bFA01D7B85Be733465C3b2BFCa96B2c4'),
             ('abi',
              '[{"inputs": [{"internalType": "address", "name": "_singleton", "type": "address"}], "stateMutability": "nonpayable", "type": "constructor"}, {"stateMutability": "payable", "type": "fallback"}]'),
             ('last_updated',
              datetime.datetime(2025, 1, 13, 0, 50, 34, 313111))])

In [5]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import partial

async def process_all_contracts():
    # Create a thread pool - adjust max_workers as needed
    thread_pool = ThreadPoolExecutor(max_workers=10)
    loop = asyncio.get_event_loop()
    
    # Create tasks that run in thread pool
    tasks = [
        loop.run_in_executor(
            thread_pool,
            partial(asyncio.run, processor._process_contract(contract['contract_address'], json.loads(contract['abi'])))
        )
        for contract in contract_abis
    ]
    
    # Gather results
    results = await asyncio.gather(*tasks)
    contract_info = [result for result in results if result]
    for result in contract_info:
        print(result)
    
    thread_pool.shutdown()
    return contract_info

# Run the async function

In [6]:
contract_info = await process_all_contracts()

Error processing contract <web3._utils.datatypes.Contract object at 0x0000020E82893850>: ("The function 'factory' was not found in this ", "contract's abi.")
Error processing contract <web3._utils.datatypes.Contract object at 0x0000020ECAEF71D0>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x0000020E82892F10>: ("The function 'factory' was not found in this ", "contract's abi.")
Error processing contract <web3._utils.datatypes.Contract object at 0x0000020E828945D0>: ("The function 'factory' was not found in this ", "contract's abi.")
Error processing contract <web3._utils.datatypes.Contract object at 0x0000020E8952BD50>: ("The function 'factory' was not found in this ", "contract's abi.")
Error processing contract <web3._utils.datatypes.Contract object at 0x0000020EFE4625D0>: ("The function 'factory' was not found in this ", "contract's abi.")
E

ContractInfo(address='0x42659Ea5dEe757ce0F7CAE37c93db3E148756db5', factory='0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f', fee=None, token0_name='FusionX AI', token1_name='Wrapped Ether', name=None)
ContractInfo(address='0x9d9681d71142049594020bD863D34D9F48d9DF58', factory=('0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f',), fee=None, token0_name='Wrapped Ether', token1_name='Dogs Of Elon', name=None)
ContractInfo(address='0x54578B6F942aEB23b67A8ceF24220651306B8e26', factory='0x1F98431c8aD98523631AE4a59f267346ea31F984', fee=3000, token0_name='HuntToken', token1_name='Tether USD', name=None)
ContractInfo(address='0xF6C3595cBd6858B47E93c83312CEF89750cEA3A4', factory='0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f', fee=None, token0_name='Wrapped Ether', token1_name='Somnium Space Cubes', name=None)
ContractInfo(address='0x7b3Ed3A3444144ff526554D5F4Aa463D1759f51a', factory='0x1F98431c8aD98523631AE4a59f267346ea31F984', fee=10000, token0_name='Wrapped Ether', token1_name='AxonDAO Governance Token', n